# How do I _create_, _start_, & _monitor_ a task?
### Overview
We are getting into advanced techniques here and will need to leverage a few other cookbooks. You will need an app and some files in your project, then it is easy to start one. The beginning of this notebook **replicates** the <a href="tasks_create.ipynb"> tasks_create </a> notebook.


### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within one of your projects (we will just use that call directly here). This is a **great place** to get the **app_id** you will need in this recipe.
 5. You have at least one app in your project, maybe from <a href="apps_copyFromPublicApps.ipynb" target="_blank">copying one</a>
 6. You may want to review how to <a href="apps_detailOne.ipynb" target="_blank"> get details </a> of your app (we will assume you do, and pass the appropriate inputs).

 
### WARNING
This will burn through some processing credits (**about \$0.50**). You can create _DRAFT_ tasks to just see how it works, swap the commenting in **Build and run tasks** to only run: 
```python
# task created in DRAFT state
task = api.tasks.create(name=task_name, project=my_project.id, app=single_app.id, inputs=inputs, run=False)       

# task created and RUN immediately
task = api.tasks.create(name=task_name, project=my_project.id, app=single_app.id, inputs=inputs, run=True)       
```

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.  
We also import the `Conflict` error to handle situations when trying to copy over an existing app.

In [ ]:
import sevenbridges as sbg
from sevenbridges.errors import Conflict

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbpla, etc}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Find your project 
First, we identify an **interesting project** (by _name_) by searching though all of our projects<sup>1</sup>

In [ ]:
# [USER INPUT] Set project and App names:
# Note that you can have multiple apps or projects with the same name. It is best practice to reference entities by ID.
project_id = 'user-name/my-project-name'     
a_name = 'SBG FASTA Indices'

# Get my project
my_project = api.projects.get(project_id)                       

## Copy an app to use for the task
Next, we find an **interesting app** (by _name_), again by searching though all of the apps _within_ that project<sup>1</sup>. Note, we are reusing my_project from above.

<sup>1</sup> A _cleaner_ way to do this would be to identify by project and app **id**.

In [ ]:
# Find app by name in the Public Apps
public_app = [a for a in api.apps.query(
        visibility='public', limit=100
    ).all() if a.name == a_name][0]

try:
    my_app = public_app.copy(project=my_project)
    print('App {} copied to Project {}.'.format(my_app.name, destination_project.name))
except Conflict:
    my_app = [a for a in api.apps.query(project=my_project, limit=100).all() if a.name == a_name][0]
    print('App already exists in the destination project, reusing the existing app.')
    
# re-list apps in target project to verify the copy worked
my_apps = api.apps.query(project=my_project.id, limit=100)
my_app_names = [a.name for a in my_apps.all()]

if a_name in my_app_names:
    print('Sucessfully copied or reused one app!')
else:
    print('Something went wrong...')

## Copy a public reference file to use for an input
We will first find our _source\_project_ (the Public Reference Files), then list the files within the source project<sup>2</sup>, and copy a file from **_source\_project_ -> _my\_project_**.

<sup>2</sup> Files are only accessible **within** a project - here the Public Reference project (**warning** we may change this project name in the future).

In [ ]:
# [USER INPUT] Set file and Public Reference Project names here:
source_project_id = 'admin/sbg-public-data'        
f_name = 'ucsc.hg19.fasta'              


# LIST all file names in target project
my_files = [f.name for f in api.files.query(limit=100, project=my_project.id).all()]
# Find source file
source_file = api.files.query(limit=100, project=source_project_id, names=[f_name])

# Make sure that file exists in Public Reference
if not source_file:
    print("File ({}) does not exist in Public Reference, please check spelling".format(f_name))
    raise KeyboardInterrupt
else:
    source_file = source_file[0]
    
# Check if first file already exists in the target project
if source_file.name in my_files:
    print('File already exists in second project, using that one')
    my_new_file = api.files.query(limit=100, project=my_project.id, names=[source_file.name])[0]
else:
    print('File {} does not exist in Project {}; copying now'.format(source_file.name, my_project.id))
    my_new_file = source_file.copy(project=my_project.id,
                                   name=source_file.name)

    # re-list files in target project to verify the copy worked
    my_files = [f.name for f in api.files.query(limit=100, project=my_project.id).all()]
    
    if source_file.name in my_files:
        print('Sucessfully copied one file!')
    else:
        print('Something went wrong...')

## Check app inputs

First we can check what inputs are expected in our app. The app property `raw` contains a `dict` that has all the app information ready for grabs. Going through the `raw['inputs]'` can show us expected input IDs and types.
Hints for input types: 
- The `[]` suffix, e.g. `File[]`, is a syntax denoting a list of items, in this case Files. 
- The `?` suffix, e.g. `File?`, is a syntax denoting an optional input, in this case an optional File. Another way to represent an optional type is, e.g. `['null', 'int']` for optional integer input.

In [ ]:
print('Expected inputs for app {}:'.format(my_app.name))
for inp in my_app.raw['inputs']:
    print('id: {}{}type: {}'.format(inp['id'].lstrip('#'), ' ' * (30 - len(inp['id'])), inp['type']))

## Create & start the task
Here we use the reference file and set one of the optional configuration inputs. Note that input files are passed a _file_ (or a _list_ of _files_) while configuration parameters are passed just the values.

Note that you need funds in your project billing group in order to run the task or you can get a TaskValidationError. You can check out task errors with `my_task.errors`

In [ ]:
# Task description
task_name = 'task created with task_create.ipynb'
inputs = {'reference':my_new_file} # 'fasta' is a 'File_Inputs'

# if your task has any 'Config_inputs' that can be specified by
#    {'id':value}      # value can be str, bool, float, etc depending on tool

# Create and RUN a task
my_task = api.tasks.create(name=task_name, project=my_project.id, 
                           app=my_app.id, inputs=inputs, run=True)

## Print task status
Here we poll the recently created task. 

In [ ]:
my_task.reload()
print('Your task is in {} status'.format(my_task.status))

## Wait for task completion
Simple loop to ping for task completion.

In [ ]:
# [USER INPUT] Set loop time (seconds):
import time

loop_time = 30
flag = {'taskRunning': True}

print('Pinging SBPLAT for task completion.')
while flag['taskRunning']:

    my_task.reload()
    if my_task.status == 'COMPLETED':
        flag['taskRunning'] = False
        print('Task has completed, life is beautiful')
    elif my_task.status  == 'FAILED':  
        print('Task failed, can not continue')
        raise KeyboardInterrupt
    else:
        print('Task status: {}'.format(my_task.status))
        time.sleep(loop_time)

## Get task outputs
Here we reload the task and print task outputs. 

In [ ]:
my_task.reload()
print(my_task.outputs)

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-new-task) and [here](http://docs.sevenbridges.com/docs/perform-an-action-on-a-specific-task)